# Run this notebook on the Demand forecast cluster

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f


## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
# calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
# inbound_2022_dry = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

### Get holidays

In [0]:
flemish_holidays = (
  school_holidays()
  .filter(
    (f.col('is_dutch_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('flemish_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

walloon_holidays =(
  school_holidays()
  .filter(
    (f.col('is_french_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('walloon_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

holidays = pd.concat((flemish_holidays, walloon_holidays))


## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-09-30')#('2022-09-30')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    
}
# variables 
#pred_period = 1
# end_horizon = 7
# fc_freq=1

horizon = 8 #length of the forecasting
freq_forecast = 1
expected_pred = 1
# future_dates=1

training_period= 120

### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_df[['Dates', 'Dry Actuals']].copy() #original filtered dataset, no sundays&NANs

inbound_2022_train['x_7days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_1)
inbound_2022_train['x_14days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_2)
inbound_2022_train['x_21days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_3)

# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=expected_pred) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  
  print(f"Backtesting training ending on: {end_train_date}") 

  inbound_2022_dry_lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{inbound_2022_dry_lgbm_train.shape} ")
  # Convert Dates column to datetime
  inbound_2022_dry_lgbm_train['Dates'] = pd.to_datetime(inbound_2022_dry_lgbm_train['Dates'])
    
  inbound_2022_dry_lgbm_train = inbound_2022_dry_lgbm_train.dropna()
  features_values_train=inbound_2022_dry_lgbm_train.iloc[:, -3:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = inbound_2022_dry_lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date +dt.timedelta(days=horizon), periods= expected_pred, freq='D')
                                                                 
  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df
  prediction_dates = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration, predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=expected_pred)
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-05-03 00:00:00
print the shape:(97, 5) 
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 81
[LightGBM] [Info] Number of data points in the train set: 76, number of used features: 3
[LightGBM] [Info] Start training from score 65239.065789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

In [0]:
backtest_results

,ds,yhat
104,2022-05-11,70412.346984
105,2022-05-12,51325.776874
106,2022-05-13,61376.001404
107,2022-05-14,66476.343536
108,2022-05-16,90515.112555
...,...,...
293,2022-12-27,63051.997030
294,2022-12-28,56421.903502
295,2022-12-29,63928.868758
296,2022-12-30,53841.410752


### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-05-11,49313.0,63909.213527,70412.346984
1,2022-05-12,43377.0,59648.343564,51325.776874
2,2022-05-13,46556.0,57664.833847,61376.001404
3,2022-05-14,77061.0,59931.051337,66476.343536
4,2022-05-16,102662.0,101870.520364,90515.112555
...,...,...,...,...
188,2022-12-27,63764.0,71459.270000,63051.997030
189,2022-12-28,47680.0,71916.320000,56421.903502
190,2022-12-29,39552.0,60346.346000,63928.868758
191,2022-12-30,61119.0,52203.888000,53841.410752


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i],
                      fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge


,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-05-11,49313.0,63909.213527,70412.346984,14596.213527,21099.346984,14596.213527,21099.346984,2.130494e+08,4.451824e+08,0.295991,True,0.427866,True
1,2022-05-12,43377.0,59648.343564,51325.776874,16271.343564,7948.776874,16271.343564,7948.776874,2.647566e+08,6.318305e+07,0.375115,True,0.183249,True
2,2022-05-13,46556.0,57664.833847,61376.001404,11108.833847,14820.001404,11108.833847,14820.001404,1.234062e+08,2.196324e+08,0.238612,True,0.318326,True
3,2022-05-14,77061.0,59931.051337,66476.343536,-17129.948663,-10584.656464,17129.948663,10584.656464,2.934351e+08,1.120350e+08,0.222291,True,0.137354,True
4,2022-05-16,102662.0,101870.520364,90515.112555,-791.479636,-12146.887445,791.479636,12146.887445,6.264400e+05,1.475469e+08,0.007710,False,0.118319,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2022-12-27,63764.0,71459.270000,63051.997030,7695.270000,-712.002970,7695.270000,712.002970,5.921718e+07,5.069482e+05,0.120684,True,0.011166,False
189,2022-12-28,47680.0,71916.320000,56421.903502,24236.320000,8741.903502,24236.320000,8741.903502,5.873992e+08,7.642088e+07,0.508312,True,0.183345,True
190,2022-12-29,39552.0,60346.346000,63928.868758,20794.346000,24376.868758,20794.346000,24376.868758,4.324048e+08,5.942317e+08,0.525747,True,0.616325,True
191,2022-12-30,61119.0,52203.888000,53841.410752,-8915.112000,-7277.589248,8915.112000,7277.589248,7.947922e+07,5.296331e+07,0.145865,True,0.119072,True


In [0]:

days_out_range= backtest_df_merge.groupby('Dates').agg({'Outside_range_Dry_LGBM':'sum'}).reset_index()
days_out_range

,Dates,Outside_range_Dry_LGBM
0,2022-05-11,1
1,2022-05-12,1
2,2022-05-13,1
3,2022-05-14,1
4,2022-05-16,1
...,...,...
188,2022-12-27,0
189,2022-12-28,1
190,2022-12-29,1
191,2022-12-30,1


In [0]:
filter_df = days_out_range[days_out_range['Outside_range_Dry_LGBM'] != 0]
filter_df

,Dates,Outside_range_Dry_LGBM
0,2022-05-11,1
1,2022-05-12,1
2,2022-05-13,1
3,2022-05-14,1
4,2022-05-16,1
...,...,...
187,2022-12-26,1
189,2022-12-28,1
190,2022-12-29,1
191,2022-12-30,1


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()


In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,1.930000e+02,1.930000e+02,193.000000,193.000000,193,193
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114,150
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,64937.870466,66842.844234,62844.260212,1904.973768,-2093.610255,6606.340242,17400.628405,9.929778e+07,6.051879e+08,0.123761,0.277613,NaN,NaN
7,std,23841.744819,21405.118101,10131.342516,9806.483857,24575.066878,7479.566087,17435.053409,3.363776e+08,1.587439e+09,0.216895,0.323816,NaN,NaN
8,min,20444.000000,35712.572000,42968.239008,-59204.840000,-123821.500368,4.000000,163.851768,1.600000e+01,2.684740e+04,0.000081,0.002997,NaN,NaN
9,25%,49605.000000,54571.413926,55423.933912,-2530.120000,-13331.136410,2163.796000,6846.385110,4.682013e+06,4.687299e+07,0.033877,0.118319,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')